In [ ]:
#1
# объявление основных переменных/функций


import os
import os.path
from google.colab import output as cou
from google.colab import drive
import re
from re import search
import subprocess
from subprocess import getoutput
import threading
from IPython.display import HTML, Javascript, display, clear_output
from ipython_genutils.py3compat import builtin_mod
import ipywidgets as widgets
import requests
import glob
import time
from datetime import timedelta
import distutils.dir_util
from IPython.utils import capture
import multiprocessing
import random
import string
from tqdm import tqdm
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle

cfg = "/content/.config/"
#переменная !$dl для сокращения команды загрузки, перед использованием обязательно задать переменную пути $path
path = '/content/'
wget = "wget -nv -t 10 --show-progress --progress=bar -q --content-disposition "
dl = f"{wget} -P {path} "
sf = "https://huggingface.co/utnah/safetensors/resolve/main/"
pt = "https://huggingface.co/utnah/ckpt/resolve/main/"
esrgan = "https://huggingface.co/utnah/esrgan/resolve/main/"
lora = "https://huggingface.co/utnah/lora/resolve/main/"
hyper = "https://huggingface.co/utnah/hypernetworks/resolve/main/"
dl_deps = "wget -q -t 10 --content-disposition https://huggingface.co/utnah/deps/resolve/main/"
deps = "https://huggingface.co/utnah/deps/resolve/main/"
gh = "https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/"
dl_gh = f"wget -q -t 10 --content-disposition  {gh}"
end = '/content/.config/end'
deps_bar=f"{gh}depsprogressbar" # прогрессбар во время распаковки пакетов: loader(deps)
inprogress=f"{gh}loader_anim" # заглушка на css во время выполнения чего-либо: loader(inprogress)
import requests
m = HTML(requests.get(gh+'st').text)
ok = ""
with capture.capture_output() as stored_variables:
  %store cfg
  %store wget
  %store sf
  %store pt
  %store esrgan
  %store lora
  %store hyper
  %store dl_deps
  %store deps
  %store gh
  %store dl_gh
  %store end
  %store deps_bar
  %store inprogress
  %store m

def loader(html): # для подгрузки html/svg
  display(HTML(html, metadata=dict(isolated=True)))  

#простая качалка (для загрузки архива с зависимостями)
def download(url: str, fname: str, desc: str, chunk_size=1024):
  try:
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
      desc=desc,
      total=total,
      unit='iB',
      unit_scale=True,
      unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=chunk_size):
          size = file.write(data)
          bar.update(size)
  except KeyboardInterrupt:
    print("\nзагрузка отменена")
  except Exception as error:
    print(f"загрузка не удалась:\n{error}")

def ng():
  !$dl_gh'end' -O /content/.config/end # svg-баннер после завершения
  !$dl_gh'zrpmdNELTAkW' -O /content/.config/cfg # нгрок-токены

def depinst(): #распаковка зависимостей (которые ставятся через pip)
  #!$dl_deps'd'
  #download(f"{deps}d", "/content/d", "зависимости")
  !pip uninstall torchvision -y
  !pip uninstall torch -y
  clear()
  #loader(deps_bar)
  #with open(os.path.join("/content/.config", "deps"), "w") as deps:
  #    deps.write("\n".join(["https://huggingface.co/utnah/deps/resolve/main/deps.7z.{:03d}".format(i) for i in range(1, 13)]))
  #!cat /content/.config/deps | xargs -P 100 -n1 wget -P /content/.config -nv -t 10 --show-progress --progress=bar -q --content-disposition
  #!7z -bso0 -bd -mmt4 -slp -y x /content/.config/deps.7z.001 -o/usr/local/lib/python3.9/dist-packages
  #!rm /content/.config/deps*

def flara(): # запрос в клаудфлару, пока не выдаст ссылку
  import os
  from subprocess import getoutput
  while True:
    !cloudflared tunnel --url localhost:7860 > clf.txt 2>&1 &
    !sleep 9
    !grep -m2 -o 'https[^ ]*' /content/stable-diffusion-webui/clf.txt | tail -n1 > clfr.txt
    srvs = getoutput('cat /content/stable-diffusion-webui/clf.txt')
    if "has been created" in srvs:
      break
    !rm /content/clf.txt
    !rm /content/clfr.txt
  cflare = getoutput('cat /content/stable-diffusion-webui/clfr.txt')

#проигрывание аудио, чтобы вкладка не убивалась
def mobile_player():
  cou.eval_js("var audio = new Audio('https://github.com/PR0LAPSE/StableDiffusionWebUIColab/raw/main/src/psyattack.wav'); audio.addEventListener('ended', function() { audio.currentTime = 0; audio.play(); }, false); audio.play();")

# css-файл для внедрения в виджеты
css_setup_widget = requests.get(f"{gh}css_setup_widget.html").text
def setup_css():
  display(HTML(f'''
    {css_setup_widget}
'''))
  

# установка времени запуска колаба для таймера
try:
  launching
except:
  launching = int(time.time())-6
  with capture.capture_output() as stored_variables:
    %store launching

# автоматическое открытие ссылок градио и нгрок
def auto_opener():
  display(HTML('''
  <script>
  var linkOpened = false;
  function checkPage() {
    if (linkOpened) {
      return;
    }
    var links = document.getElementsByTagName("a");
    for (var i = 0; i < links.length; i++) {
      if (links[i].href.indexOf("gradio") !== -1) {
        window.open(links[i].href, '_blank');
        var ngroklink = document.querySelector('[href*="ngrok"]');
        window.open(ngroklink, '_blank');
        linkOpened = true;
        return;
      }
    }
  }
  checkPage();
  var observer = new MutationObserver(function(mutations) {
    mutations.forEach(function(mutation) {
      checkPage();
    });
  });
  observer.observe(document.body, { childList: true, subtree: true });
  </script>
  '''))

# совет пользователям Firefox
def check_browser():
  display(HTML('''
  <script>
  var isFirefox = typeof InstallTrigger !== 'undefined';
  var isGecko = navigator.userAgent.toLowerCase().indexOf('gecko') !== -1 && navigator.userAgent.toLowerCase().indexOf('webkit') === -1;
  var isFirefoxGecko = isFirefox || isGecko;
  var os = navigator.platform.toLowerCase();
  if (isFirefoxGecko) {
    const injectCSS = css => {
    let el = document.createElement('style');
    el.type = 'text/css';
    el.innerText = css;
    document.head.appendChild(el);
    return el;};
    injectCSS('@keyframes blink {0% {color: #ff5732;}50% {color: #eb0c0c;}100% {color: #ff5732;}}#non-chrome {position: absolute;z-index: 999;top: 0;left: 0;width: 100%;height: 100%;background-color: rgba(0, 0, 0, 0.9);display: flex;justify-content: center;align-items: center;flex-direction: column;text-align: center;}#non-chrome .browser_alarm {animation: blink 1s infinite;font-size: 1.8em }#non-chrome .expirince {font-size: 1.55em;}#non-chrome p {color: #fff }#non-chrome a {font-size: 1.55em;padding: 0 1em;color: #308df7;}#close-link {margin-top: 2em;color: #3d5d81!important;text-decoration: none;}');
    var message = document.createElement('div');
    message.id = 'non-chrome';
    document.body.appendChild(message);
    var osMessage = '';
    if (os.indexOf('win') !== -1) {
      osMessage = ' Windows';
      urls = '<p><a target="_blank" href="https://dl.google.com/update2/installers/ChromeSetup.exe">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/Thorium-Win/releases">Thorium</a> | <a target="_blank" href="https://github.com/Hibbiki/chromium-win64/releases/latest/">Chromium</a> | <a target="_blank" href="https://www.microsoft.com/ru-ru/edge/download">Edge</a></p>'
    } else if (os.indexOf('mac') !== -1) {
      osMessage = 'macOS';
      urls = '<p><a target="_blank" href="https://dl.google.com/chrome/mac/universal/dev/googlechromedev.dmg">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/Thorium-Special/releases/tag/M109.0.5414.120-2">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com/#mac-64-bit">Chromium</a> | <a target="_blank" href="https://www.microsoft.com/RU-ru/edge">Edge</a></p>'
    } else if (os.indexOf('linux') !== -1) {
      osMessage = 'Linux';
      urls = '<p><a target="_blank" href="https://www.google.com/intl/ru/chrome/dev/">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/thorium/releases/tag/M110.0.5481.178">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com/#linux-64-bit">Chromium</a> | <a target="_blank" href="https://packages.microsoft.com/repos/edge/pool/main/m/microsoft-edge-stable/">Edge</a></p>'
    } else {
      osMessage = os;
      urls = '<p><a target="_blank" href="https://www.google.com/intl/ru/chrome/dev/">Chrome</a> | <a target="_blank" href="https://github.com/Alex313031/thorium">Thorium</a> | <a target="_blank" href="https://chromium.woolyss.com>Chromium</a> | <a target="_blank" href="https://www.microsoft.com/ru-ru/edge/download">Edge</a></p>'
    }
    var browserAlarm = '<h2 class="browser_alarm">Firefox и браузеры на его движке могут испытывать проблемы с поддержкой в '+osMessage+'</h2>';
    message.innerHTML = browserAlarm + '<p class="expirince">Для наилучшего пользовательского опыта в Google Colab установите браузер на движке Chromium:</p>' + urls + '<a href="#" id="close-link">закрыть</a>';
    var closeLink = document.getElementById('close-link');
    closeLink.addEventListener('click', function(e) {
      e.preventDefault();
      message.style.display = 'none';
    });
  }
  </script>
  '''))